In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files


# Install necessary libraries
!pip install keras gradio huggingface_hub

import numpy as np
from huggingface_hub import from_pretrained_keras
from PIL import Image
import io
import keras
import gradio as gr

# Load the model



Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled hugg

In [11]:
import gradio as gr
from PIL import Image
import numpy as np
import keras
from keras.models import load_model
import cv2
from skimage.metrics import structural_similarity as ssim

# Load the pre-trained model
model = from_pretrained_keras("keras-io/lowlight-enhance-mirnet", compile=False)

# Function to calculate PSNR
def calculate_psnr(original, enhanced):
    return cv2.PSNR(original, enhanced)

# Function to calculate SSIM
def calculate_ssim(original, enhanced):
    original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_RGB2GRAY)
    return ssim(original_gray, enhanced_gray)

# Define a function for processing images
def enhance_image(uploaded_image):
    try:
        # Open and preprocess the image
        low_light_img = Image.open(uploaded_image).convert('RGB')
        original_image = np.array(low_light_img)  # Keep the original for metrics
        original_size = low_light_img.size  # Get the original size
        low_light_img = low_light_img.resize((256, 256), Image.NEAREST)
        image = keras.preprocessing.image.img_to_array(low_light_img)
        image = image.astype('float32') / 255.0
        image = np.expand_dims(image, axis=0)

        # Model prediction
        output = model.predict(image)

        # Post-process the output
        output_image = output[0] * 255.0
        output_image = output_image.clip(0, 255)
        output_image = output_image.reshape((np.shape(output_image)[0], np.shape(output_image)[1], 3))
        output_image = np.uint8(output_image)

        # Resize the output image to match the original dimensions
        output_image = Image.fromarray(output_image, 'RGB')
        #output_image = output_image.resize(original_size, Image.BICUBIC)  # Resize to original dimensions

        # Save the output image
        #output_image=output_image.resize((64*64), Image.BICUBIC)
        output_path = 'enhanced_image.png'
        output_image.save(output_path)

        #output_image = Image.fromarray(output_image, 'RGB')
        output_image = output_image.resize(original_size, Image.BICUBIC)
        # Resize to original dimensions
        # Calculate PSNR and SSIM
        psnr_value = calculate_psnr(original_image, np.array(output_image))
        ssim_value = calculate_ssim(original_image, np.array(output_image))

        # Create metrics string
        metrics = f"PSNR: {psnr_value:.2f} dB\nSSIM: {ssim_value:.4f}\n"

        # Save metrics to a text file
        metrics_path = 'org_details.txt'
        with open(metrics_path, 'w') as f:
            f.write(metrics)

        # Return the path to the enhanced image and metrics for display
        return output_path, metrics_path

    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Define the Gradio interface
with gr.Blocks() as interface:
    # Input for uploading an image
    image_input = gr.Image(type="filepath", label="Upload a Low-Light PNG Image")
    # Outputs for displaying the enhanced image
    image_output = gr.Image(type="filepath", label="Enhanced Image")
    # Textbox for displaying metrics
    metrics_output = gr.Textbox(label="Metrics")

    # Button to enhance the image
    enhance_button = gr.Button("Enhance Image")
    enhance_button.click(fn=enhance_image, inputs=image_input, outputs=[image_output, metrics_output])

# Launch the interface
interface.launch()


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27508cd7f382d875f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
